In [1]:
import pandas as pd
import numpy as np
# pd.set_option("display.show_dimensions", False)
# pd.set_option("display.float_format", "{:4.2g}".format)

### 改变DataFrame的形状

In [2]:
#pd读取之后取部分列赋值soils
soils = pd.read_csv("Soils.csv", index_col=0)[["Depth", "Contour", "Group", "pH", "N"]]
soils_mean = soils.groupby(["Depth", "Contour"]).mean()
%C 4 soils.head(); soils_mean.head()

            soils.head()                           soils_mean.head()            
------------------------------------    ----------------------------------------
   Depth Contour  Group    pH      N                      Group      pH        N
1   0-10     Top      1  5.40  0.188    Depth Contour                           
2   0-10     Top      1  5.65  0.165    0-10  Depression    9.0  5.3525  0.17825
3   0-10     Top      1  5.14  0.260          Slope         5.0  5.5075  0.21900
4   0-10     Top      1  5.14  0.169          Top           1.0  5.3325  0.19550
5  10-30     Top      2  5.14  0.164    10-30 Depression   10.0  4.8800  0.08025
                                              Slope         6.0  5.2825  0.10100


#### 添加删除列或行

In [3]:
#列操作评估
soils["N_percent"] = soils.eval("N * 100")
soils.head()

,Depth,Contour,Group,pH,N,N_percent
1,0-10,Top,1,5.40,0.188,18.8
2,0-10,Top,1,5.65,0.165,16.5
3,0-10,Top,1,5.14,0.260,26.0
4,0-10,Top,1,5.14,0.169,16.9
5,10-30,Top,2,5.14,0.164,16.4


In [4]:
#assign方法会返回新的df而不改变原df
soils2=soils.assign(pH2 = soils.pH + 1)
%C 4 soils.head();soils2.head()

                  soils.head()                                         soils2.head()                    
-----------------------------------------------    -----------------------------------------------------
   Depth Contour  Group    pH      N  N_percent       Depth Contour  Group    pH      N  N_percent   pH2
1   0-10     Top      1  5.40  0.188       18.8    1   0-10     Top      1  5.40  0.188       18.8  6.40
2   0-10     Top      1  5.65  0.165       16.5    2   0-10     Top      1  5.65  0.165       16.5  6.65
3   0-10     Top      1  5.14  0.260       26.0    3   0-10     Top      1  5.14  0.260       26.0  6.14
4   0-10     Top      1  5.14  0.169       16.9    4   0-10     Top      1  5.14  0.169       16.9  6.14
5  10-30     Top      2  5.14  0.164       16.4    5  10-30     Top      2  5.14  0.164       16.4  6.14


In [5]:
np.random.randint(10, 20)

19

In [6]:
def random_dataframe(n):
    np.random.seed(5) #初始随机数，让结果稳定
    columns = ["A", "B", "C"]
    for i in range(n):
        nrow = np.random.randint(10, 20)
        yield pd.DataFrame(np.random.randint(0, 100, size=(nrow, 3)), columns=columns)
        #yield将返回生成器，只有在调用其中的元素的时候才会运算，可用next进行调用
df_list = list(random_dataframe(1000))
df_list[0]

,A,B,C
0,78,61,16
1,73,8,62
2,27,30,80
3,7,76,15
4,53,80,27
5,44,77,75
6,65,47,30
7,84,86,18
8,9,41,62
9,1,82,16


In [7]:
#append方法被弃用
# %%time 
# df_res1 = pd.DataFrame([])
# for df in df_list:
#     df_res1 = df_res1.append(df)

In [8]:
%%time
df_res2 = pd.concat(df_list, axis=0)
df_res2.head()

CPU times: total: 78.1 ms
Wall time: 79.3 ms


,A,B,C
0,78,61,16
1,73,8,62
2,27,30,80
3,7,76,15
4,53,80,27


In [11]:
#key关键字将作为0级index传入
df_res3 = pd.concat(df_list, axis=0, keys=list(range(len(df_list))))
df_res3.loc[30].equals(df_list[30]) #合成df中0级index为30的数据块与原dflist中的第30个元素相比较


True

In [12]:
#删除指定行列
soils.drop(["N", "Group"], axis=1).head()

,Depth,Contour,pH,N_percent
1,0-10,Top,5.40,18.8
2,0-10,Top,5.65,16.5
3,0-10,Top,5.14,26.0
4,0-10,Top,5.14,16.9
5,10-30,Top,5.14,16.4


#### 行索引与列之间相互转换

In [17]:
#将index转换为列
soils_mean.reset_index(level="Contour").head()

,Contour,Group,pH,N
Depth,,,,
0-10,Depression,9.0,5.3525,0.17825
0-10,Slope,5.0,5.5075,0.21900
0-10,Top,1.0,5.3325,0.19550
10-30,Depression,10.0,4.8800,0.08025
10-30,Slope,6.0,5.2825,0.10100


In [20]:
#将列转为index,append为False时将删除原索引
soils_mean.set_index("Group", append=True).head()

pH        N
Depth Contour    Group                 
0-10  Depression 9.0    5.3525  0.17825
      Slope      5.0    5.5075  0.21900
      Top        1.0    5.3325  0.19550
10-30 Depression 10.0   4.8800  0.08025
      Slope      6.0    5.2825  0.10100

#### 行和列的索引相互转换

In [21]:
#行索引转换为列索引
soils_mean.unstack(1)[["Group", "pH"]].head()

Group                    pH                
Contour Depression Slope  Top Depression   Slope     Top
Depth                                                   
0-10           9.0   5.0  1.0     5.3525  5.5075  5.3325
10-30         10.0   6.0  2.0     4.8800  5.2825  4.8500
30-60         11.0   7.0  3.0     4.3625  4.2675  4.2050
60-90         12.0   8.0  4.0     4.1725  3.9275  3.8925

In [22]:
#当所有索引被转到一个轴上后将得到一个Series
soils_mean.stack().head(10)

Depth  Contour          
0-10   Depression  Group     9.00000
                   pH        5.35250
                   N         0.17825
       Slope       Group     5.00000
                   pH        5.50750
                   N         0.21900
       Top         Group     1.00000
                   pH        5.33250
                   N         0.19550
10-30  Depression  Group    10.00000
dtype: float64

#### 交换索引的等级

In [26]:
#交换索引的级别并排序，排序可以让0级索引相同的值排到一块
soils_mean.swaplevel(0, 1).sort_index()

Group      pH        N
Contour    Depth                        
Depression 0-10     9.0  5.3525  0.17825
           10-30   10.0  4.8800  0.08025
           30-60   11.0  4.3625  0.05050
           60-90   12.0  4.1725  0.04025
Slope      0-10     5.0  5.5075  0.21900
           10-30    6.0  5.2825  0.10100
           30-60    7.0  4.2675  0.06075
           60-90    8.0  3.9275  0.04300
Top        0-10     1.0  5.3325  0.19550
           10-30    2.0  4.8500  0.11750
           30-60    3.0  4.2050  0.07950
           60-90    4.0  3.8925  0.05775

#### 透视表

In [29]:
#生成透视表
df = soils_mean.reset_index()[["Depth", "Contour", "pH", "N"]]
df_pivot_pH = df.pivot("Depth", "Contour", "pH")
%C 4 df; df_pivot_pH

                  df                                  df_pivot_pH            
--------------------------------------    -----------------------------------
    Depth     Contour      pH        N    Contour  Depression   Slope     Top
0    0-10  Depression  5.3525  0.17825    Depth                              
1    0-10       Slope  5.5075  0.21900    0-10         5.3525  5.5075  5.3325
2    0-10         Top  5.3325  0.19550    10-30        4.8800  5.2825  4.8500
3   10-30  Depression  4.8800  0.08025    30-60        4.3625  4.2675  4.2050
4   10-30       Slope  5.2825  0.10100    60-90        4.1725  3.9275  3.8925
5   10-30         Top  4.8500  0.11750                                       
6   30-60  Depression  4.3625  0.05050                                       
7   30-60       Slope  4.2675  0.06075                                       
8   30-60         Top  4.2050  0.07950                                       
9   60-90  Depression  4.1725  0.04025                          

In [46]:
#透视表若不指定value参数，则会将所有的剩余列进行透视
df.pivot("Depth", "Contour")

pH                          N                  
Contour Depression   Slope     Top Depression    Slope      Top
Depth                                                          
0-10        5.3525  5.5075  5.3325    0.17825  0.21900  0.19550
10-30       4.8800  5.2825  4.8500    0.08025  0.10100  0.11750
30-60       4.3625  4.2675  4.2050    0.05050  0.06075  0.07950
60-90       4.1725  3.9275  3.8925    0.04025  0.04300  0.05775

In [47]:
#逆透视
df_before_melt = df_pivot_pH.reset_index()
df_after_melt = pd.melt(df_before_melt, id_vars="Depth", value_name="pH")
%C df_before_melt; df_after_melt

              df_before_melt                        df_after_melt        
------------------------------------------  -----------------------------
Contour  Depth  Depression   Slope     Top      Depth     Contour      pH
0         0-10      5.3525  5.5075  5.3325  0    0-10  Depression  5.3525
1        10-30      4.8800  5.2825  4.8500  1   10-30  Depression  4.8800
2        30-60      4.3625  4.2675  4.2050  2   30-60  Depression  4.3625
3        60-90      4.1725  3.9275  3.8925  3   60-90  Depression  4.1725
                                            4    0-10       Slope  5.5075
                                            5   10-30       Slope  5.2825
                                            6   30-60       Slope  4.2675
                                            7   60-90       Slope  3.9275
                                            8    0-10         Top  5.3325
                                            9   10-30         Top  4.8500
                                      